In [35]:
from re import sub
from typing import List, Dict

def unify(e1: str, e2: str, substitution: Dict[str, str] = None) -> Dict[str, str]:
    """
    Performs unification of two first-order logic expressions.

    Args:
        e1: The first expression.
        e2: The second expression.
        substitution: The current substitution.

    Returns:
        A dictionary representing the substitution if unification is successful, None otherwise.
    """
    if substitution is None:
        substitution = {}

    # Apply the current substitution to both expressions
    e1 = apply_substitution(e1, substitution)
    e2 = apply_substitution(e2, substitution)

    # If expressions are identical, no further substitution is needed
    if e1 == e2:
        return substitution

    # If either expression is a variable, unify it
    if is_variable(e1):
        return unify_variable(e1, e2, substitution)

    if is_variable(e2):
        return unify_variable(e2, e1, substitution)

    # If both are functions, try to unify the function symbols and arguments
    if is_function(e1) and is_function(e2):
        func_symbol1 = get_function_symbol(e1)
        func_symbol2 = get_function_symbol(e2)

        # If function symbols don't match, unification fails
        if func_symbol1 != func_symbol2:
            return None

        # Unify the function arguments pairwise
        args1 = get_function_arguments(e1)
        args2 = get_function_arguments(e2)

        if len(args1) != len(args2):
            return None

        for arg1, arg2 in zip(args1, args2):
            substitution = unify(arg1, arg2, substitution)
            if substitution is None:
                return None

        return substitution

    # If none of the above, the unification fails
    return None


def unify_variable(var, expr, substitution):
    """
    Unifies a variable with an expression.
    If the variable is already substituted, recursively unify the substituted expression.
    """
    # If the variable is already substituted, apply the substitution recursively
    if var in substitution:
        return unify(substitution[var], expr, substitution)

    # If the expression is already substituted, apply the substitution recursively
    if expr in substitution:
        return unify(var, substitution[expr], substitution)

    # Check if unifying the variable with the expression would result in a circular definition
    if is_function(expr) and contains_variable(expr, var):
        return None

    # Add the variable to the substitution map
    new_substitution = substitution.copy()
    new_substitution[var] = expr
    return new_substitution


def apply_substitution(expr, substitution):
    """
    Applies the current substitution to an expression.
    """
    if is_variable(expr):
        return substitution.get(expr, expr)
    elif is_function(expr):
        func_symbol = get_function_symbol(expr)
        args = get_function_arguments(expr)
        new_args = [apply_substitution(arg, substitution) for arg in args]
        return f"{func_symbol}({','.join(new_args)})"
    else:
        return expr


def is_variable(expr):
    """
    Checks if an expression is a variable (single lowercase letter).
    """
    return expr.islower() and expr.isalpha()


def is_function(expr):
    """
    Checks if an expression is a function (i.e., it contains parentheses).
    """
    return "(" in expr and ")" in expr


def get_function_symbol(expr):
    """
    Extracts the function symbol from a function expression.
    Assumes the expression is valid and contains parentheses.
    """
    return expr[:expr.index("(")]


def get_function_arguments(expr):
    """
    Extracts the arguments of a function expression.
    Assumes the expression is valid and contains parentheses.
    """
    args_str = expr[expr.index("(") + 1:expr.index(")")]
    return [arg.strip() for arg in args_str.split(',')]


def contains_variable(expr, var):
    """
    Checks if a given function expression contains a variable.
    """
    # Check all arguments of the function for the variable
    if is_function(expr):
        args = get_function_arguments(expr)
        return any(contains_variable(arg, var) for arg in args)
    return var in expr


# Example Usage: Success
expr1 = "owns(x,y)"
expr2 = "owns(Ram,bike)"
substitution = unify(expr1, expr2)
if substitution is None:
  substitution = 'failure'
print(f"Unification of {expr1} and {expr2}: {substitution}")  # Output: {'x': 'a', 'y': 'b'}
print('\n')
# Example usage: Failure due to circular variable
expr1 = 'owns(x,bike(y))'
expr2 = 'owns(Ram,car(audi))'
substitution = unify(expr1, expr2)
if substitution is None:
  substitution = 'failure'
print(f"Unification of {expr1} and {expr2}: {substitution}")  # Output: failure


Unification of owns(x,y) and owns(Ram,bike): {'x': 'Ram', 'y': 'bike'}


Unification of owns(x,bike(y)) and owns(Ram,car(audi)): failure
